In [52]:
# Необходимо построить классификатор изображений рукописного ввода на базе MNIST. 
# В качестве шаблона можно использовать ipython-ноутбук 002-digit.ipynb.
# Классификатор предлагается строить на признаках, полученных в результате предобработки 
# изображений, например, гистограммы градиентов (HOG) или результат PCA преобразования.
# В качестве модели классификатора можно использовать любую известную модель, за исключением 
# сверточных нейронных сетей.

# **Критерии качества **
# Метрика accuracy. Чтобы получить зачёт по работе работе, значение метрики должно быть 
# больше 0.6. Метрика оценивается на тестовой выборке в рамках контеста Digit Recognizer на Kaggle.

# Как сдавать
# Решение необходимо предоставить в виде** ipytnon-ноутбука** с реализацией процесса 
# построения модели и скриншота с финальным результатом на Kaggle.

In [53]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [54]:
# read train
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
sub = pd.read_csv('../input/sample_submission.csv')
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [55]:
sub.shape

(28000, 2)

In [56]:
labels = train.label

In [57]:
features = train.drop('label', axis=1)

In [58]:
from sklearn.model_selection import train_test_split
y_train, y_val, x_train, x_val = train_test_split(
    labels, features, test_size=0.2,
    random_state=42)

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [61]:
imp = pd.Series(clf.feature_importances_)
imp = imp.sort_values(ascending=False)
imp = imp[imp.values>=0.008667]
#print(imp)
print(train.columns[imp.index])

Index(['pixel405', 'pixel374', 'pixel238', 'pixel514', 'pixel567', 'pixel378',
       'pixel433', 'pixel353', 'pixel459', 'pixel377'],
      dtype='object')


In [62]:
cols_to_use = train.columns[imp.index]

In [63]:
cols_to_use

Index(['pixel405', 'pixel374', 'pixel238', 'pixel514', 'pixel567', 'pixel378',
       'pixel433', 'pixel353', 'pixel459', 'pixel377'],
      dtype='object')

In [64]:
redused_features = train[cols_to_use]

In [65]:
classfier = RandomForestClassifier()
classfier.fit(x_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [66]:
pred_val = classfier.predict(x_val)

In [67]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_val, pred_val))

[[803   0   3   1   2   2   3   0   2   0]
 [  0 898   2   1   1   1   2   0   3   1]
 [  7   6 778  15   9   2   7  11   8   3]
 [  1   5  19 853   0  16   1  13  20   9]
 [  5   1   5   1 791   1  10   2   9  14]
 [  5   1   1  19   2 654  10   3   3   4]
 [  9   1   6   1   6  11 749   0   2   0]
 [  1   5  22   2  11   1   0 834   1  16]
 [  2   8  13  12  12  16   6   2 757   7]
 [  3   3   4  16  22   3   1  20   6 760]]


In [68]:
from sklearn.metrics import accuracy_score
print('Accuracy: %s' % accuracy_score(y_val, pred_val))

Accuracy: 0.9377380952380953
